In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! gdown https://drive.google.com/uc?id=1oij5U35z8qe_cA1LNuDw882dQU1jWmC9
! unzip histology-images-query-competition.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: train/train/30cc5f140e59.png  
  inflating: train/train/30cdaeb5b1ca.png  
  inflating: train/train/30cf3345ee12.png  
  inflating: train/train/30dd693d43d6.png  
  inflating: train/train/30f75d19dfc4.png  
  inflating: train/train/3107722cc068.png  
  inflating: train/train/310ae29da47b.png  
  inflating: train/train/3113ac127dbc.png  
  inflating: train/train/3116b2b0f304.png  
  inflating: train/train/312484622415.png  
  inflating: train/train/312f579464b6.png  
  inflating: train/train/313ac9feff13.png  
  inflating: train/train/313e2705a8a9.png  
  inflating: train/train/3152b3228ea1.png  
  inflating: train/train/3153317bf8fb.png  
  inflating: train/train/315936a5e347.png  
  inflating: train/train/3159ce5c615b.png  
  inflating: train/train/316a5962ae4c.png  
  inflating: train/train/3185995a0689.png  
  inflating: train/train/318d4afd39f8.png  
  inflating: train/train/31936c3b67a3.png  
  inflating: train/train/3199c87cc25e.png  
  inflating

In [3]:
! gdown https://drive.google.com/uc?id=1CBAM-L2j2qKLJ2jXGAMC4SD9ZMwIgZSo
! gdown https://drive.google.com/uc?id=171edMp03GqaRb6jVyVCM6mUq30pOUPZG

Downloading...
From: https://drive.google.com/uc?id=1CBAM-L2j2qKLJ2jXGAMC4SD9ZMwIgZSo
To: /content/byol.py
100% 8.10k/8.10k [00:00<00:00, 3.52MB/s]
Downloading...
From: https://drive.google.com/uc?id=171edMp03GqaRb6jVyVCM6mUq30pOUPZG
To: /content/validation_ground_truth.csv
100% 5.22k/5.22k [00:00<00:00, 9.92MB/s]


In [4]:
import PIL
from tqdm import tqdm
import os
import pandas as pd
import math
import time
import shutil
import random

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torch.utils.data import Dataset,DataLoader

from byol import BYOL

In [5]:
# !pip install https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

# # ! export XRT_TPU_CONFIG="tpu_worker;0;$TPU_IP_ADDRESS:8470"

# # imports pytorch
# import torch

# # imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm

# device = xm.xla_device()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
class ValidDataSet(Dataset):
    def __init__(self, images_folder_path, label_dic, transform=None):
        self.images_folder_path = images_folder_path
        self.label_dic = label_dic
        if transform is not None:
          self.transform=transform
        else:
          self.transform=transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.label_dic)
    def __getitem__(self, index):
        element = self.label_dic.iloc[index]
        predic = element['prediction']
        query = element['query']
        img_name=query.split('_')
        images=[]
        # print(img_name)
        for i in range(2):
          images.append(PIL.Image.open(os.path.join(
              self.images_folder_path, img_name[i]+'.png')))
          images[i] = self.transform(images[i])
          
        return images[0],images[1], predic

class TestDataSet(Dataset):
    def __init__(self, images_folder_path, pair_dic, transform=None):
        self.images_folder_path = images_folder_path
        self.pair_dic = pair_dic
        if transform is not None:
          self.transform=transform
        else:
          self.transform=transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.pair_dic)
    def __getitem__(self, index):
        element = self.pair_dic.iloc[index]
        img_name=(element[0],element[1])
        label=str(element[0].split('.')[0]+'_'+element[1].split('.')[0])

        images=[]
        # print(img_name)
        for i in range(2):
          images.append(PIL.Image.open(os.path.join(
              self.images_folder_path, img_name[i])))
          images[i] = self.transform(images[i])
          
        return images[0],images[1], label

In [7]:
import time

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def train(train_loader, model, optimizer,scheduler, epoch,print_freq):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()

    for images,_ in tqdm(train_loader):
      data_time.update(time.time() - end)

      # images[0] = images[0].to(device)
      # images[1] = images[1].to(device)

      loss = model([images[0].to(device),images[1].to(device)])
      losses.update(loss.item(), images[0].size(0))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step()

      model.update_moving_average() # update moving average of target encoder

      # measure elapsed time
      batch_time.update(time.time() - end)
      end = time.time()

      # if i % print_freq == 0:
      #       progress.display(i)
    torch.save(model.state_dict(), f'checkpoint_{epoch}.pth')
    print(losses)
    
def valid(valid_loader,model):
    # losses = AverageMeter('Loss', ':.4f')
    # progress = ProgressMeter(
    #     len(train_loader),
    #     [losses],
    #     prefix="Epoch: [{}]".format(epoch))
    valid_loss=0
    predicts = []

    threshold=20
    for i in range(threshold):
      predicts.append([[0,0],[0,0]])

    model.eval()
    with torch.no_grad():
      for images1,images2,labels in tqdm(valid_loader):
          loss = model([images1.to(device),images2.to(device)],predict=True)

          # print(loss)
          for i in range(len(labels)):
            for j in range(threshold):
              # print(float(j/10))
              predicts[j][labels[i]][loss<float(j/10)]+=1
            valid_loss+=loss if labels[i] else 1-loss
      print(f'valid_loss:{valid_loss}')
      for i in range(threshold):
        print(f'predict:{predicts[i]}')

def test(test_loader,model,threshold):

    predicts = []

    model.eval()
    with torch.no_grad():
      for images1,images2,label in tqdm(test_loader):
          loss = model([images1.to(device),images2.to(device)],predict=True)
          predicts.append((label[0],1 if loss<threshold else 0))
    return predicts
    
      

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [8]:

class TwoCropsTransform:
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        q = self.base_transform(x)
        k = self.base_transform(x)
        return [q, k]

class GaussianBlur(object):
    """Gaussian blur augmentation in SimCLR https://arxiv.org/abs/2002.05709"""

    def __init__(self, sigma=[.1, 2.]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = x.filter(PIL.ImageFilter.GaussianBlur(radius=sigma))
        return x

train_augmentation = [
        transforms.RandomResizedCrop(256,scale=(0.5,1.0)),
        transforms.RandomRotation(45),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5),
        # transforms.RandomAdjustSharpness(2, p=0.5),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.62 , 0.45 ,0.65 ],
          std=[0.1386 ,0.176 ,0.168])
    ]

test_augmentation=[
        transforms.Resize(256),
        # transforms.RandomApply([
        #     transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
        # ], p=0.8),
        # transforms.RandomGrayscale(p=0.2),
        # transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5),
        # transforms.RandomAdjustSharpness(2, p=1.0),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.62 , 0.45 ,0.65 ],
          std=[0.1386 ,0.176 ,0.168])
]


divide_transform=TwoCropsTransform(transforms.Compose(train_augmentation))
test_transform=transforms.Compose(test_augmentation)

In [9]:
batch_size=32
workers=2

train_dataset = datasets.ImageFolder(
        'train/',transform=divide_transform)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,num_workers=workers, pin_memory=True, sampler=None, drop_last=True)


valid_csv=pd.read_csv('validation_ground_truth.csv')
valid_set=ValidDataSet('train/train',valid_csv,transform=test_transform)
valid_loader=DataLoader(valid_set, num_workers=workers, pin_memory=True)

In [11]:
resnet = models.resnet50(pretrained=True)

model = BYOL(
    resnet,
    image_size = 256,
    hidden_layer = 'avgpool'
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)


save_dir='drive/MyDrive/byol_checkpoint'

model.load_state_dict(torch.load(save_dir+'/checkpoint_{:04d}.pth.tar'.format(19))['state_dict'])

model.to(device)

epoches=20


scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, eta_min=1e-5, T_max=20)

In [ ]:


for epoch in range(epoches):
    print(f'epoce: {epoch}')
    train(train_loader,model,optimizer,scheduler,epoch,10)
    valid(valid_loader,model)
    save_checkpoint({
                # 'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                # 'optimizer' : opt.state_dict(),
            }, is_best=False, filename=save_dir+'/checkpoint_{:04d}.pth.tar'.format(epoch))

# save your improved network
torch.save(resnet.state_dict(), './improved-net.pt')

epoce: 0


100%|██████████| 193/193 [07:27<00:00,  2.32s/it]


Loss 0.1682 (0.1578)


100%|██████████| 186/186 [00:39<00:00,  4.76it/s]


valid_loss:-37.786163330078125
predict:[[114, 0], [72, 0]]
predict:[[114, 0], [71, 1]]
predict:[[114, 0], [71, 1]]
predict:[[114, 0], [68, 4]]
predict:[[114, 0], [68, 4]]
predict:[[114, 0], [62, 10]]
predict:[[114, 0], [58, 14]]
predict:[[113, 1], [56, 16]]
predict:[[112, 2], [55, 17]]
predict:[[110, 4], [51, 21]]
predict:[[109, 5], [47, 25]]
predict:[[105, 9], [43, 29]]
predict:[[104, 10], [39, 33]]
predict:[[103, 11], [32, 40]]
predict:[[101, 13], [30, 42]]
predict:[[98, 16], [30, 42]]
predict:[[91, 23], [26, 46]]
predict:[[86, 28], [24, 48]]
predict:[[84, 30], [17, 55]]
predict:[[76, 38], [16, 56]]
epoce: 1


...

100%|██████████| 193/193 [07:27<00:00,  2.32s/it]


Loss 0.1191 (0.1426)


100%|██████████| 186/186 [00:38<00:00,  4.77it/s]


valid_loss:-38.974327087402344
predict:[[114, 0], [72, 0]]
predict:[[114, 0], [71, 1]]
predict:[[114, 0], [71, 1]]
predict:[[114, 0], [68, 4]]
predict:[[114, 0], [68, 4]]
predict:[[114, 0], [65, 7]]
predict:[[114, 0], [61, 11]]
predict:[[114, 0], [58, 14]]
predict:[[113, 1], [56, 16]]
predict:[[112, 2], [51, 21]]
predict:[[111, 3], [48, 24]]
predict:[[109, 5], [43, 29]]
predict:[[108, 6], [38, 34]]
predict:[[104, 10], [34, 38]]
predict:[[101, 13], [31, 41]]
predict:[[98, 16], [30, 42]]
predict:[[93, 21], [25, 47]]
predict:[[87, 27], [21, 51]]
predict:[[79, 35], [17, 55]]
predict:[[77, 37], [16, 56]]
epoce: 13


 13%|█▎        | 26/193 [01:01<06:25,  2.31s/it]